# Лабораторная работа 

# №7. Дискретное преобразрвание Фурье

## Писарик Максим, 5130901/10101

Импортируем модуль thinkdsp.py.

In [3]:
import os

if not os.path.exists('thinkdsp.py'):
    !wget https://github.com/AllenDowney/ThinkDSP/raw/master/code/thinkdsp.py

In [4]:
import numpy as np
PI2 = 2 * np.pi

## УПРАЖНЕНИЕ 7.2

В данном упражнении необходимо реализовать БПФ по следующему рекурсивному алгоритму:
    1) Массив сигнала y разделить на четные элементы е и нечетные элементы о;
    2) Вычислить ДПФ е и о, делая рекурсивные вызовы;
    3) Вычислить ДПФ для каждого значения n, используя теорему Дэниелсона-Ланцоша.

Для начала, вычислим БПФ для небольшого сигнала с помощью np.fft.fft:

In [5]:
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
hs

array([ 0.2+0.j , -1.2-0.2j,  0.2+0.j , -1.2+0.2j])

Затем, вычислим ДПФ:

In [6]:
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

In [7]:
hs2 = dft(ys)

Затем сравним результаты шагов 1 и 2:

In [8]:
np.sum(np.abs(hs - hs2))

5.864775846765962e-16

Можем видеть, что результаты совпадают (есть ошибка округления).

Далее сделаем нерекурсивную функцию, которая разбивает массив на 2 части (четные и нечетные элементы) и вычисляет БПФ для каждой из них.

In [9]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    return np.tile(He, 2) + W * np.tile(Ho, 2)

Применим функцию и снова сравним результаты:

In [10]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))

0.0

Можем убедиться, что результаты совпадают.

Затем добавим базовый случай и рекурсивные вызовы.

In [11]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys
    He = fft(ys[::2])
    Ho = fft(ys[1::2])
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    return np.tile(He, 2) + W * np.tile(Ho, 2)

Применим функцию и сравним результаты:

In [12]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))

1.6653345369377348e-16

Результаты совпадают (есть ошибка округления). Время работы данного алгоритма пропорционально NlogN.